input: map_phrase.json {map: [phrases]}

output: rdf ttl file (mapRDF.ttl)


USGS-15-CA-hesperia-e1902-s1898-rp1912: empty data, so not used

In [1]:
import json

with open('map_phrase.json', 'r') as f:
    phrase_dic = json.load(f)

In [2]:
# the number of distinct phrase per each map
for key in phrase_dic:
    print(len(phrase_dic[key]))

In [3]:
from rdflib import Graph, URIRef, Literal, XSD, Namespace
from rdflib.namespace import RDF

In [4]:
# http://www.ldf.fi/service/rdf-grapher

MYNS = Namespace('gs:kg:')
schema = Namespace('http://schema.org/')
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
geo = Namespace("http://www.opengis.net/ont/geosparql#")
geoname = Namespace("http://linkedgeodata.org/ontology/")

my_kg = Graph()
my_kg.bind('my_ns', MYNS)
my_kg.bind('schema', schema)
my_kg.bind('geo', geo)
my_kg.bind('rdfs', rdfs)
my_kg.bind('geoname', geoname)

In [5]:
for key in phrase_dic:
    map_name = key
    node_uri = URIRef(MYNS["mapName:"+map_name])
    my_kg.add((node_uri, rdfs['type'], schema['Map']))
    my_kg.add((node_uri, geoname['nearby'], Literal(key+": Name nearby:addLater")))
    
    i = 0
    for phrase in phrase_dic[key]:
        i += 1
        feature_uri = URIRef(MYNS[key+'feature_id'+str(i)])
        my_kg.add((node_uri, geo['sfOverlaps'], feature_uri))

        my_kg.add((feature_uri, rdfs['label'], Literal(phrase)))
        my_kg.add((feature_uri, rdfs['seeAlso'], URIRef(key+"externalDBuri"+str(i))))

        my_kg.add((feature_uri, geo['asWKT'], Literal(key+": addLater"+str(i))))


In [6]:
my_kg.serialize('mapRDF.ttl', format="turtle")